In [8]:
import ee

import sys 
sys.path.append('../geoindexity')

In [3]:
# In order to use GEE (its EO data collections AND computing resources) we have to authentificate
# and to initialize the project we want to work in. 
ee.Authenticate()
ee.Initialize()

In [3]:
class time_series:
    """
    The core geoindexity time series object. 
    ...

    Attributes
    ----------
    aoi : list
        [xmin, ymin, xmax, ymax]
    startdate : str
        'yyyy-mm-dd'
    enddate : str
        'yyyy-mm-dd'
    product: str 
        GEE data set: 'S1', 'S2_TOA', 'S2_SR' 
    cloudcover: int 
        percentage cloudcover threshold for series
    collection: ee.imagecollection.ImageCollection 
        the actual image collection as input for index 
        time-series

    Methods
    -------
    fetch_data(self):
        Fetches GEE Image collection based on the following time_series attributes:
        product, aoi, startdate, enddate, cloudcover

    """
    def __init__(self, aoi, startdate, enddate, product, cloudcover):
        data_dict = {'S1': 'COPERNICUS/S1_GRD', 
             'S2_TOA': 'COPERNICUS/S2_HARMONIZED',
             'S2_SR': 'COPERNICUS/S2_SR_HARMONIZED'}
        
        self.aoi = ee.Geometry.Rectangle(aoi)
        self.startdate = startdate
        self.enddate = enddate
        self.product = data_dict[product] 
        self.cloudcover = cloudcover
        self.collection = None

    def __len__(self): 
        if self.collection != None:
            return self.collection.size().getInfo()
        else: 
            return 0

    def fetch_data(self): 
        """
        Fetches GEE Image collection based on the following time_series attributes:
        product, aoi, startdate, enddate, cloudcover
        ...
        Images with cloud percentage <= cloudcover are used for the collection
        """
        ee_collection = ee.ImageCollection(self.product)
        self.collection = ee_collection.filterBounds(self.aoi).filterDate(self.startdate, self.enddate).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', self.cloudcover))    

In [4]:
aoi = [-2.9,
       47.48,
       -2.6,
       47.63
      ]

ts = time_series(aoi,
            startdate='2023-06-01',
            enddate='2023-08-30', 
            product= 'S2_SR', 
            cloudcover=10)

In [5]:
len(ts) 

0

In [6]:
ts.fetch_data()

In [7]:
len(ts)

7

In [8]:
type(ts.collection)

ee.imagecollection.ImageCollection

## Testing 26/06

In [9]:
from config import *

In [10]:
class Sentinel:
    """
    A class that represents the user speci
    """
    def __init__(self, roi, start_date, end_date, properties=None):
        self.roi = roi
        self.start_date = start_date
        self.end_date = end_date
        self.collection_id= 'COPERNICUS/S2_SR_HARMONIZED'
        self.properties = properties or {}

    def bound(self):
        return ee.Geometry.Rectangle(self.roi)

    def select_product(self):
        image_collection = ee.ImageCollection(self.collection_id)
        image_collection = image_collection.filterDate(self.start_date, self.end_date)
        image_collection = image_collection.filterBounds(self.bound())
        if self.properties:
            valid_properties = {"CLOUDY_PIXEL_PERCENTAGE": int, "CLOUDY_SHADOW_PERCENTAGE": int, "DARK_FEATURES_PERCENTAGE": int}
            for property_name, value in self.properties.items():
                if property_name not in valid_properties:
                    raise KeyError(f"Invalid property name: {property_name}. Supported properties: {', '.join(valid_properties.keys())}.")
                image_collection=image_collection.filter(ee.Filter.lte(property_name, value))
        return image_collection

    def number_of_images(self):
        print(f'Total Sentinel images collected: {self.select_product().size().getInfo()}')

In [32]:
class Landsat8:
    """
    A class that represents the user speci
    """
    def __init__(self, roi, start_date, end_date, properties=None):
        self.roi = roi
        self.start_date = start_date
        self.end_date = end_date
        self.collection_id= 'LANDSAT/LC08/C02/T1_L2'
        self.properties = properties or {}

    def bound(self):
        return ee.Geometry.Rectangle(self.roi)

    def select_product(self):
        image_collection = ee.ImageCollection(self.collection_id)
        image_collection = image_collection.filterDate(self.start_date, self.end_date)
        image_collection = image_collection.filterBounds(self.bound())
        if self.properties:
            valid_properties = {"CLOUDY_PIXEL_PERCENTAGE": int, "CLOUDY_SHADOW_PERCENTAGE": int, "DARK_FEATURES_PERCENTAGE": int}
            for property_name, value in self.properties.items():
                if property_name not in valid_properties:
                    raise KeyError(f"Invalid property name: {property_name}. Supported properties: {', '.join(valid_properties.keys())}.")
                image_collection=image_collection.filter(ee.Filter.lte(property_name, value))
        return image_collection

class Geoindexity:
    """
    The core geoindexity time series object. 
    ...

    Attributes
    ----------
    aoi : list
        [xmin, ymin, xmax, ymax]
    startdate : str
        'yyyy-mm-dd''
    enddate : str
        'yyyy-mm-dd'
    collection: str 
        default: 'Sentinel', 'Landsat' 
    properties: dict 
        percentage cloudcover threshold for series

    Methods
    -------
    fetch_data(self):
        Fetches GEE Image collection based on the following time_series attributes:
        product, aoi, startdate, enddate, cloudcover

    """
    # Will be changed accordingly to the config file parameters

    def __init__(self, roi, start_date, end_date, collection_id='Sentinel', properties=None): # collection argument as identifier between Sentinel and Landsat? 
        self.roi = roi
        self.start_date = start_date
        self.end_date = end_date
        self.collection_id = collection_id
        self.properties = properties


        if self.collection_id == 'Sentinel':
            self.collection = Sentinel(roi=self.roi,
                                       start_date=self.start_date,
                                       end_date=self.end_date,
                                       properties=self.properties
                                       ).select_product()

        if self.collection_id == 'Landsat8':
            self.collection = Landsat8(roi=self.roi,
                                       start_date=self.start_date,
                                       end_date=self.end_date,
                                       properties=self.properties
                                       ).select_product()

    def __len__(self):
        return self.collection.size().getInfo()

    def bound(self):
        return ee.Geometry.Rectangle(self.roi)

    def add_ndvi(self, image):
        if self.collection_id == 'Sentinel':
            ndvi = image.expression(
                '((nir-red)/(nir+red))',
                {'nir': image.select('B4'),
                 'red': image.select('B8')}
            ).rename('NDVI')
        elif self.collection == 'Landsat8':
            ndvi = image.expression(
                '((nir - red)/(nir + red))',
                {'nir': image.select('SR_B5'),
                 'red': image.select('SR_B4')}
            ).rename('NDVI')

        return image.addBands(ndvi)


    def add_evi(self, image):
        if self.collection == 'Sentinel':
            evi = image.expression(
                '2.4*((nir-red)/(nir+ 6*red - 7.5* blue +1))',
                {'nir': image.select('SR_B8'),
                 'red': image.select('SR_B4'),
                 'blue': image.select('SR_B2')}
            ).rename('EVI')


        elif self.collection == 'Landsat8':
            evi = image.expression(
                '(2.5 * (nir–red) / (nir + 6 * red – 7.5 * blue + 1)', # Change to Landsat8 here!
                {'nir': image.select('SR_B5'),
                 'red': image.select('SR_B4'),
                 'blue': image.select('SR_B2')}
            ).rename('EVI')

        return image.addBands(evi)

    def ndvi_collection(self):
        self.collection = self.collection.map(self.add_ndvi)

    def evi_collection(self):
        self.collection = self.collection.map(self.add_evi)


In [44]:
x = Geoindexity(roi, start_date, end_date, collection_id='Sentinel')

In [33]:
y = Geoindexity(roi, start_date, end_date, collection_id='Landsat8')

In [41]:
def test_index(index, geoindexity): 
    if index in geoindexity.collection.first().bandNames().getInfo():
        print('JUHUU')
    else: 
        print('MEEH')

In [45]:
test_index('NDVI', x)

MEEH


In [46]:
x.ndvi_collection()
test_index('NDVI', x)

JUHUU


In [47]:
y.ndvi_collection()
test_index('NDVI', y)

UnboundLocalError: cannot access local variable 'ndvi' where it is not associated with a value

UnboundLocalError only for Landast8 - NDVI

In [48]:
x.evi_collection()
test_index('EVI', x)

UnboundLocalError: cannot access local variable 'evi' where it is not associated with a value

In [49]:
y.evi_collection()
test_index('EVI', y)

UnboundLocalError: cannot access local variable 'evi' where it is not associated with a value